Learning notes from: https://realpython.com/python-pandas-tricks/

key points:

1. Configure Options & Settings at Interpreter Startup
2. ~~Make Toy Data Structures With Pandas’ Testing Module~~
3. Take Advantage of Accessor Methods
4. Create a DatetimeIndex From Component Columns
5. Use Categorical Data to Save on Time and Space
6. Introspect Groupby Objects via Iteration
7. Use This Mapping Trick for Membership Binning
8. ~~Understand How Pandas Uses Boolean Operators~~
9. Load Data From the Clipboard
10. Write Pandas Objects Directly to Compressed Format

In [1]:
import pandas as pd
import numpy as np

## 1. Configure Options & Settings at Interpreter Startup

Python [The Interactive Startup File](https://docs.python.org/3/tutorial/appendix.html#the-interactive-startup-file). Set a env variable to the startup script and refer it in all other scripts. (Seems a lot of work)

startup script 

```python
import pandas as pd

def start():
    options = {
        'display': {
            'max_columns': None,
            'max_colwidth': 25,
            'expand_frame_repr': False,  # Don't wrap to multiple pages
            'max_rows': 14,
            'max_seq_items': 50,         # Max length of printed sequence
            'precision': 4,
            'show_dimensions': False
        },
        'mode': {
            'chained_assignment': None   # Controls SettingWithCopyWarning
        }
    }

    for category, option in options.items():
        for op, value in option.items():
            pd.set_option(f'{category}.{op}', value)  # Python 3.6+

if __name__ == '__main__':
    start()
    del start  # Clean up namespace in the interpreter
```

referencing scripts

```python
import os
filename = os.environ.get('PYTHONSTARTUP')
if filename and os.path.isfile(filename):
    with open(filename) as fobj:
        startup_file = fobj.read()
    exec(startup_file)
```


IPython [Startup Files](https://ipython.readthedocs.io/en/stable/interactive/tutorial.html#startup-files). `~/.ipython/profile_default/startup/`

## 3. Take Advantage of Accessor Methods

- .str maps to StringMethods.
- .dt maps to CombinedDatetimelikeProperties.
- .cat routes to CategoricalAccessor.

### .str

In [2]:
pd.Series._accessors

{'cat', 'dt', 'sparse', 'str'}

In [3]:
addr = pd.Series([
    'Washington, D.C. 20003',
    'Brooklyn, NY 11211-1755',
    'Omaha, NE 68154',
    'Pittsburgh, PA 15211'
])

In [4]:
addr.str.upper()

0     WASHINGTON, D.C. 20003
1    BROOKLYN, NY 11211-1755
2            OMAHA, NE 68154
3       PITTSBURGH, PA 15211
dtype: object

In [5]:
addr.str.count(r'\d')

0    5
1    9
2    5
3    5
dtype: int64

In [6]:
# extract regex to different columns
regex = (r'(?P<city>[A-Za-z ]+), '      # One or more letters
         r'(?P<state>[A-Z]{2}) '        # 2 capital letters
         r'(?P<zip>\d{5}(?:-\d{4})?)')  # Optional 4-digit extension

addr.str.replace('.', '').str.extract(regex)

,city,state,zip
0,Washington,DC,20003
1,Brooklyn,NY,11211-1755
2,Omaha,NE,68154
3,Pittsburgh,PA,15211


### .dt

In [7]:
daterng = pd.Series(pd.date_range('2017', periods=9, freq='Q'))
daterng.dt.day_name()

0      Friday
1      Friday
2    Saturday
3      Sunday
4    Saturday
5    Saturday
6      Sunday
7      Monday
8      Sunday
dtype: object

In [8]:
daterng[daterng.dt.quarter > 2]

2   2017-09-30
3   2017-12-31
6   2018-09-30
7   2018-12-31
dtype: datetime64[ns]

In [9]:
daterng[daterng.dt.is_year_end]

3   2017-12-31
7   2018-12-31
dtype: datetime64[ns]

## 4. Create a DatetimeIndex From Component Columns

In [10]:
from itertools import product
datecols = ['year', 'month', 'day']
df = pd.DataFrame(
    list(product([2017, 2016], [1, 2], [1, 2, 3])),
    columns=datecols,
)
df['data'] = np.random.randn(len(df))
df

,year,month,day,data
0,2017,1,1,1.681722
1,2017,1,2,-2.669073
2,2017,1,3,0.479715
3,2017,2,1,0.119712
4,2017,2,2,0.661939
5,2017,2,3,0.452302
6,2016,1,1,-0.697892
7,2016,1,2,0.538994
8,2016,1,3,0.191025
9,2016,2,1,0.411914


In [11]:
# create datetime index
df.index = pd.to_datetime(df[datecols])
df.head()

,year,month,day,data
2017-01-01,2017,1,1,1.681722
2017-01-02,2017,1,2,-2.669073
2017-01-03,2017,1,3,0.479715
2017-02-01,2017,2,1,0.119712
2017-02-02,2017,2,2,0.661939


In [12]:
# drop redundant columns and convert a single column dataframe to a series
df = df.drop(datecols, axis=1).squeeze()
df.head()

2017-01-01    1.681722
2017-01-02   -2.669073
2017-01-03    0.479715
2017-02-01    0.119712
2017-02-02    0.661939
Name: data, dtype: float64

In [13]:
# check dtype of the index
df.index.dtype_str

'datetime64[ns]'

## 5. Use Categorical Data to Save on Time and Space

In [14]:
# create a string series
colors = pd.Series([
    'periwinkle',
    'mint green',
    'burnt orange',
    'periwinkle',
    'burnt orange',
    'rose',
    'rose',
    'mint green',
    'rose',
    'navy'
])

In [15]:
# repeat the series
manycolors = colors.repeat(10)
len(manycolors) / manycolors.nunique()

20.0

In [16]:
# memory usage
manycolors.memory_usage(index=False, deep=True)

6500

In [17]:
# save memory by using `Categorical`
manycolors.astype('category').memory_usage(index=False, deep=True)

585

In [40]:
# convert to Categorical
ccolors = colors.astype('category')
ccolors.cat.categories

Index(['burnt orange', 'mint green', 'navy', 'periwinkle', 'rose'], dtype='object')

In [19]:
# encoded
ccolors.cat.codes

0    3
1    1
2    0
3    3
4    0
5    4
6    4
7    1
8    4
9    2
dtype: int8

In [20]:
# methods
[i for i in dir(ccolors.cat) if not i.startswith('_')]

['add_categories',
 'as_ordered',
 'as_unordered',
 'categorical',
 'categories',
 'codes',
 'index',
 'name',
 'ordered',
 'remove_categories',
 'remove_unused_categories',
 'rename_categories',
 'reorder_categories',
 'set_categories']

In [21]:
# less fexible
ccolors.iloc[5] = 'a new color'

ValueError: Cannot setitem on a Categorical with a new category, set the categories first

In [41]:
# NOTE: must add to categories first!!
ccolors = ccolors.cat.add_categories(['a new color'])
ccolors.iloc[5] = 'a new color'

## 6. Introspect Groupby Objects via Iteration

`pandas.core.groupby.generic.DataFrameGroupBy` is an interable object, which returns an iterator.

This can be very helpful if you want to do a series of operations to each group.

In [32]:
url = ('https://archive.ics.uci.edu/ml/'
       'machine-learning-databases/abalone/abalone.data')
cols = ['sex', 'length', 'diam', 'height', 'weight', 'rings']
abalone = pd.read_csv(url, usecols=[0, 1, 2, 3, 4, 8], names=cols)

In [33]:
abalone['ring_quartile'] = pd.qcut(abalone.rings, q=4, labels=range(1, 5))
grouped = abalone.groupby('ring_quartile')
grouped

In [34]:
for idx, frame in grouped:
    print(f"Ring quartile: {idx}".center(25,'-'))
    print(frame.nlargest(3, 'weight'), end='\n\n')

-----Ring quartile: 1----
     sex  length   diam  height  weight  rings ring_quartile
2619   M   0.690  0.540   0.185  1.7100      8             1
1044   M   0.690  0.525   0.175  1.7005      8             1
1026   M   0.645  0.520   0.175  1.5610      8             1

-----Ring quartile: 2----
     sex  length  diam  height  weight  rings ring_quartile
2811   M   0.725  0.57   0.190  2.3305      9             2
1426   F   0.745  0.57   0.215  2.2500      9             2
1821   F   0.720  0.55   0.195  2.0730      9             2

-----Ring quartile: 3----
     sex  length  diam  height  weight  rings ring_quartile
1209   F   0.780  0.63   0.215   2.657     11             3
1051   F   0.735  0.60   0.220   2.555     11             3
3715   M   0.780  0.60   0.210   2.548     11             3

-----Ring quartile: 4----
     sex  length   diam  height  weight  rings ring_quartile
891    M   0.730  0.595    0.23  2.8255     17             4
1763   M   0.775  0.630    0.25  2.7795     12 

In [35]:
# group keys
grouped.groups.keys()

dict_keys([1, 2, 3, 4])

In [36]:
# access a specific group
grouped.get_group(2).head()

,sex,length,diam,height,weight,rings,ring_quartile
2,F,0.530,0.420,0.135,0.6770,9,2
8,M,0.475,0.370,0.125,0.5095,9,2
19,M,0.450,0.320,0.100,0.3810,9,2
23,F,0.550,0.415,0.135,0.7635,9,2
39,M,0.355,0.290,0.090,0.3275,9,2


## 7. Use This Mapping Trick for Membership Binning

In [37]:
countries = pd.Series([
    'United States',
    'Canada',
    'Mexico',
    'Belgium',
    'United Kingdom',
    'Thailand'
])

# mapping
groups = {
    'North America': ('United States', 'Canada', 'Mexico', 'Greenland'),
    'Europe': ('France', 'Germany', 'United Kingdom', 'Belgium')
}

In [38]:
group_reverse = {x: k for k, v in groups.items() for x in v}
countries.map(group_reverse)

0    North America
1    North America
2    North America
3           Europe
4           Europe
5              NaN
dtype: object

## 9. Load Data From the Clipboard

When you have buffered data in clipboard

```python
df = pd.read_clipboard(na_values=[None])
```

## 10. Write Pandas Objects Directly to Compressed Format

In [39]:
# dataframe to json and compress
output_path = 'df.json.gz'

abalone.to_json(
    output_path, 
    orient='records',
    lines=True, 
    compression='gzip',
)

# remove file
import os

os.remove(output_path)